In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [36]:
Dataset = pd.read_csv("WELFake_Dataset.csv")

In [37]:
Dataset.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


## Dataset Exploration

In [38]:
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72134 entries, 0 to 72133
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  72134 non-null  int64 
 1   title       71576 non-null  object
 2   text        72095 non-null  object
 3   label       72134 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 2.2+ MB


In [39]:
Dataset.describe()

,Unnamed: 0,label
count,72134.000000,72134.000000
mean,36066.500000,0.514404
std,20823.436496,0.499796
min,0.000000,0.000000
25%,18033.250000,0.000000
50%,36066.500000,1.000000
75%,54099.750000,1.000000
max,72133.000000,1.000000


In [40]:
Dataset.isnull().sum()

Unnamed: 0      0
title         558
text           39
label           0
dtype: int64

In [41]:
Dataset.shape

(72134, 4)

## Data Cleaning

In [42]:
#Droping Unnecessory column
Dataset.drop("Unnamed: 0",axis = 1,inplace = True)

In [43]:
Dataset.head()

,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,NaN,Did they post their votes for Hillary already?,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [44]:
#Droping null values as it won't affect Dataset much 
Dataset = Dataset.dropna().reset_index(drop = True)

In [45]:
Dataset.isnull().sum()

title    0
text     0
label    0
dtype: int64

In [46]:
Dataset.shape

(71537, 3)

In [47]:
#Combine title and text into one 'content' column
#title is just headine and text is description about that news
Dataset['content'] = Dataset['title'] + " " + Dataset['text']

In [48]:
Dataset.drop(columns=['title', 'text'], inplace=True)

In [49]:
#Rearrange columns so 'content' comes first
Dataset = Dataset[['content', 'label']]

In [51]:
Cleaned_df = Dataset

In [52]:
Cleaned_df.head()

,content,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,1
1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,1
2,"Bobby Jindal, raised Hindu, uses story of Chri...",0
3,SATAN 2: Russia unvelis an image of its terrif...,1
4,About Time! Christian Group Sues Amazon and SP...,1


## Conversion of text into numerical features with TF-IDF Vectorization and Standardizing Data

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(Dataset['content'],Dataset['label'],test_size = 0.25,random_state = 7)

In [18]:
# Create TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

In [19]:
# Fit and transform the text data into TF-IDF features
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [20]:
X_train.shape,X_test.shape

((53652, 214796), (17885, 214796))

## Model Training with Logistic regression 

In [24]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression(max_iter = 1000)
logistic.fit(X_train,y_train)
y_train_pred_logistic = logistic.predict(X_train)
y_test_pred_logistic = logistic.predict(X_test)

In [26]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
print("Training Score :\n")
print(accuracy_score(y_train_pred_logistic,y_train))
print(classification_report(y_train_pred_logistic,y_train))
print(confusion_matrix(y_train_pred_logistic,y_train))
print("--------------------------------------------")
print("Testing Score :\n")
print(accuracy_score(y_test_pred_logistic,y_test))
print(classification_report(y_test_pred_logistic,y_test))
print(confusion_matrix(y_test_pred_logistic,y_test))

Training Score :

0.9626295385074182
              precision    recall  f1-score   support

           0       0.96      0.97      0.96     26098
           1       0.97      0.96      0.96     27554

    accuracy                           0.96     53652
   macro avg       0.96      0.96      0.96     53652
weighted avg       0.96      0.96      0.96     53652

[[25205   893]
 [ 1112 26442]]
--------------------------------------------
Testing Score :

0.9480570310315907
              precision    recall  f1-score   support

           0       0.94      0.95      0.95      8582
           1       0.96      0.94      0.95      9303

    accuracy                           0.95     17885
   macro avg       0.95      0.95      0.95     17885
weighted avg       0.95      0.95      0.95     17885

[[8182  400]
 [ 529 8774]]


## Model Training with Multinomial Naive Bayes

In [28]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
param_grid = {
    'alpha': [0.01, 0.1, 0.5, 1.0, 2.0]
}

In [29]:
mnb = MultinomialNB()
grid = GridSearchCV(mnb, param_grid, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

# Best parameters and score
print("Best alpha and estimator:", grid.best_params_ , grid.best_estimator_)
print("Best cross-validation accuracy:", grid.best_score_)

# Use the best model to predict
best_mnb = grid.best_estimator_
y_train_pred_mnb = best_mnb.predict(X_train)
y_test_pred_mnb = best_mnb.predict(X_test)

Best alpha and estimator: {'alpha': 0.01} MultinomialNB(alpha=0.01)
Best cross-validation accuracy: 0.8888950695752774


In [30]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
print("Training Score :\n")
print(accuracy_score(y_train_pred_mnb,y_train))
print(classification_report(y_train_pred_mnb,y_train))
print(confusion_matrix(y_train_pred_mnb,y_train))
print("--------------------------------------------")
print("Testing Score :\n")
print(accuracy_score(y_test_pred_mnb,y_test))
print(classification_report(y_test_pred_mnb,y_test))
print(confusion_matrix(y_test_pred_mnb,y_test))

Training Score :

0.9352866621933945
              precision    recall  f1-score   support

           0       0.94      0.93      0.93     26417
           1       0.93      0.94      0.94     27235

    accuracy                           0.94     53652
   macro avg       0.94      0.94      0.94     53652
weighted avg       0.94      0.94      0.94     53652

[[24631  1786]
 [ 1686 25549]]
--------------------------------------------
Testing Score :

0.8927033827229522
              precision    recall  f1-score   support

           0       0.88      0.89      0.89      8624
           1       0.90      0.89      0.90      9261

    accuracy                           0.89     17885
   macro avg       0.89      0.89      0.89     17885
weighted avg       0.89      0.89      0.89     17885

[[7708  916]
 [1003 8258]]


## Model Training with Random Forest Algorithm

In [49]:
## Hyperparameter Training
# rf_params = {"max_depth": [10,20,30,None],
#              "max_features": ['sqrt','log2',0.01,0.05],
#              "min_samples_split": [2, 5, 10],
#              "n_estimators": [50, 100]}

In [46]:
from sklearn.ensemble import RandomForestClassifier
randomforest = RandomForestClassifier(max_depth = 10,max_features = 0.05,min_samples_split = 2,n_estimators = 50)

In [47]:
# from sklearn.model_selection import RandomizedSearchCV
# random = RandomizedSearchCV(estimator=randomforest,
#                                    param_distributions = rf_params,
#                                    n_iter=20,
#                                    cv=3,
#                                    verbose=2,
#                                    n_jobs=1)
randomforest.fit(X_train, y_train)
y_train_pred_rf = randomforest.predict(X_train)
y_test_pred_rf = randomforest.predict(X_test)

In [48]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
print("Training Score :\n")
print(accuracy_score(y_train_pred_rf,y_train))
print(classification_report(y_train_pred_rf,y_train))
print(confusion_matrix(y_train_pred_rf,y_train))
print("--------------------------------------------")
print("Testing Score :\n")
print(accuracy_score(y_test_pred_rf,y_test))
print(classification_report(y_test_pred_rf,y_test))
print(confusion_matrix(y_test_pred_rf,y_test))

Training Score :

0.93519346902259
              precision    recall  f1-score   support

           0       0.91      0.96      0.93     24956
           1       0.96      0.92      0.94     28696

    accuracy                           0.94     53652
   macro avg       0.93      0.94      0.94     53652
weighted avg       0.94      0.94      0.94     53652

[[23898  1058]
 [ 2419 26277]]
--------------------------------------------
Testing Score :

0.9217780262790047
              precision    recall  f1-score   support

           0       0.89      0.94      0.92      8224
           1       0.95      0.90      0.93      9661

    accuracy                           0.92     17885
   macro avg       0.92      0.92      0.92     17885
weighted avg       0.92      0.92      0.92     17885

[[7768  456]
 [ 943 8718]]


## Stacking 

In [52]:
from sklearn.ensemble import StackingClassifier
# Define base models
base_models = [
    ('nb', MultinomialNB(alpha=0.01)),
    ('rf', RandomForestClassifier(
        max_depth=10, 
        max_features=0.05, 
        min_samples_split=2, 
        n_estimators=50
    ))
]
# Define meta-learner
meta_model = LogisticRegression( max_iter=1000)

# Stacking Classifier
stack_model = StackingClassifier(
    estimators = base_models,
    final_estimator = meta_model,
    passthrough = True,
    cv = 3,
    n_jobs = -1
)

stack_model.fit(X_train, y_train)
y_pred = stack_model.predict(X_test)

print("Stacking Model Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Stacking Model Accuracy: 0.9584008946044171
              precision    recall  f1-score   support

           0       0.96      0.95      0.96      8711
           1       0.95      0.97      0.96      9174

    accuracy                           0.96     17885
   macro avg       0.96      0.96      0.96     17885
weighted avg       0.96      0.96      0.96     17885



In [55]:
# Saving the model
import joblib
joblib.dump(stack_model, 'stack_model.joblib')
joblib.dump(vectorizer, 'vectorizer.joblib')

['vectorizer.joblib']

## Adding custom datset to the WELFake_Dataset to Enrich Fake News Examples with WhatsApp-Like Messages 

In [24]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [25]:
# Load the dataset
df = pd.read_csv("custom_fake_real_news_dataset.csv")
df.head()

,text,label
0,Drinking hot water every hour cures COVID-19 c...,0
1,Scientists discover new species of dinosaur in...,1
2,Government to ban use of mobile phones after 6...,0
3,Oxford scientists develop promising malaria va...,1
4,WHO declares COVID-19 no longer a global healt...,1


In [26]:
# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

In [27]:
# Preprocessing function
def clean_text(text):
    if pd.isnull(text):
        return ""
    text = text.lower()                              # Lowercase
    text = re.sub(r"[^a-z\s]", "", text)             # Remove punctuation/numbers
    text = re.sub(r"\s+", " ", text).strip()         # Remove extra spaces
    words = text.split()                             # Tokenization
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return " ".join(words)

In [28]:
# Combine title and text before cleaning
df["text"] = df["text"].fillna('')
df["content"] = df["text"].apply(clean_text)

In [29]:
# Droping "text" Column
df.drop("text",axis = 1,inplace = True)

In [30]:
df.head()

,label,content
0,0,drinking hot water every hour cure covid compl...
1,1,scientist discover new specie dinosaur argentina
2,0,government ban use mobile phone pm
3,1,oxford scientist develop promising malaria vac...
4,1,declares covid longer global health emergency


In [31]:
df = df[["content","label"]]
df.head()

,content,label
0,drinking hot water every hour cure covid compl...,0
1,scientist discover new specie dinosaur argentina,1
2,government ban use mobile phone pm,0
3,oxford scientist develop promising malaria vac...,1
4,declares covid longer global health emergency,1


#### Merging custom dataset and WELFake_Dataset

In [55]:
merged_df = pd.concat([Cleaned_df, df], ignore_index=True)
merged_df.tail()

,content,label
72532,moon turn green due mar magnetic pull,0
72533,un release climate change impact report,1
72534,alien landed nevada desert government hiding t...,0
72535,isro announces new moon mission,1
72536,drinking hot water every hour cure covid compl...,0


In [58]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(merged_df['content'],merged_df['label'],test_size = 0.25,random_state = 7)

In [59]:
# Create TF-IDF vectorizer
new_vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))

In [60]:
# Fit and transform the text data into TF-IDF features
X_train = new_vectorizer.fit_transform(X_train)
X_test = new_vectorizer.transform(X_test)

In [64]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [ ]:
# Define base models
base_models = [
    ('nb', MultinomialNB(alpha=0.01)),
    ('rf', RandomForestClassifier(
        max_depth=15, 
        max_features=0.05, 
        min_samples_split=5, 
        n_estimators=150
    ))
]
# Define meta-learner
meta_model = LogisticRegression(max_iter=1000)

# Stacking Classifier
new_stack_model = StackingClassifier(
    estimators = base_models,
    final_estimator = meta_model,
    passthrough = True,
    cv = 3,
    n_jobs = -1
)

new_stack_model.fit(X_train, y_train)
new_y_pred = new_stack_model.predict(X_test)


In [65]:
# Evaluating Model
print("Stacking Model Accuracy:", accuracy_score(y_test, new_y_pred))
print(classification_report(y_test, new_y_pred))

Stacking Model Accuracy: 0.9693410532120209
              precision    recall  f1-score   support

           0       0.98      0.96      0.97      8956
           1       0.96      0.98      0.97      9179

    accuracy                           0.97     18135
   macro avg       0.97      0.97      0.97     18135
weighted avg       0.97      0.97      0.97     18135



In [66]:
# Saving the model
import joblib
joblib.dump(new_stack_model, 'new_stack_model.joblib')
joblib.dump(new_vectorizer, 'new_vectorizer.joblib')

['new_vectorizer.joblib']